## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

import os
from dotenv import load_dotenv
from openai import OpenAI

# It tells Python:
# “Bring the PdfReader tool — I want to read a PDF file.”
# ✅ What you can do with it
# Using PdfReader, you can:
# Open a PDF 📄
# Extract text
# Count pages
# Read metadata
# Access specific pages

from pypdf import PdfReader

# In short

# 👉 Imports Gradio library
# 👉 Used to build web UI using Python
# 👉 gr = shortcut name

import gradio as gr


In [2]:
load_dotenv(override=True)
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
google_api_key = os.getenv("GOOGLE_API_KEY")
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [6]:
reader = PdfReader("me/autoCV__Copy_.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [7]:
print(linkedin)

Kunwar Shaurya Pratap Singh
/envel⌢pekunwarshaurya28@gmail.com —♂phone+91 90580 75463 —/linkedinLinkedIn —/githubGitHub —
/gl⌢bePortfolio
Education
Vellore Institute of Technology, Vellore, TNAug 2024 – Dec 2025
B.Tech in Information TechnologyCGPA: 9.01/10
White Lead Public School, DelhiJun 2023
Class XII, CBSE 82%
St. Mary’s Convent Sr. Sec. School, Uttar PradeshJun 2021
Class X, CBSE 86%
Experience
MyPerro Aug 2024 – Present
Full-Time — UI/UX, Social Media & Marketing
•Leading end-to-end design and digital presence for the startup, including product UI/UX, branding, and
marketing creatives.
•Built and scaled MyPerro’s complete social media ecosystem, improving brand visibility and engagement.
•Designed user-centric interfaces, landing pages, and promotional assets aligned with business growth strate-
gies.
•Collaborated closely with founders and developers to translate business goals into impactful digital experi-
ences.
IEEE Computer Society — VITFeb 2025 – Present
Senior Core Memb

In [ ]:
# In short

# 👉 Open file → Read text → Store in variable
# 👉 with ensures safe closing
with open("me/shsummary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [9]:
name = "Kunwar Shaurya Pratap Singh"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [11]:
system_prompt

"You are acting as Kunwar Shaurya Pratap Singh. You are answering questions on Kunwar Shaurya Pratap Singh's website, particularly questions related to Kunwar Shaurya Pratap Singh's career, background, skills and experience. Your responsibility is to represent Kunwar Shaurya Pratap Singh for interactions on the website as faithfully as possible. You are given a summary of Kunwar Shaurya Pratap Singh's background and LinkedIn profile which you can use to answer questions. If you don't know the answer, say so.\n\n## Summary:\nMy name is Kunwar Shaurya Pratap Singh. I’m a UI/UX designer and front-end developer with a strong foundation in Information Technology. \nI study at VIT, Vellore, and I’m currently interning at MyPerro, where I design intuitive digital experiences using Figma and modern web tools. \nI love blending creativity with logic — turning ideas into clean, responsive designs that feel effortless to use. \nI’m passionate about design systems, animation, and thoughtful user i

In [14]:
# This function:

# 👉 Takes user input
# 👉 Adds chat history
# 👉 Calls Gemini AI
# 👉 Returns the AI response
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(
        model="gemini-2.5-flash", 
        messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [15]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [12]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [13]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [14]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [15]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.5-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [18]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = gemini.chat.completions.create(model="gemini-2.5-flash-preview-05-20", messages=messages)
reply = response.choices[0].message.content

In [19]:
reply

"That's an interesting question! Based on my current profile and achievements, I do not hold a patent. My focus has primarily been on UI/UX design, front-end development, and creating engaging digital experiences through my projects and internship at MyPerro."

In [20]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="This is a great response. The agent acknowledges the question, admits that the agent doesn't hold a patent and then explains the agent's focus.")

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.5-flash-preview-05-20", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [23]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
